Trained Model at:
[https://huggingface.co/Hammad7/plag-col-rev-en-v2](https://huggingface.co/Hammad7/plag-col-rev-en-v2)

In [ ]:
import sentence_transformers
# from terminal 
# pip install sentence-transformers

/mnt/disks/user/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/mnt/disks/user/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and gui

In [ ]:
# !pip install gdown
# https://drive.google.com/file/d/12YB0YDkmLtGjqBcx400HuCSyYnvVXgj8/view?usp=sharing
!gdown --id "12YB0YDkmLtGjqBcx400HuCSyYnvVXgj8"

Downloading...
From: https://drive.google.com/uc?id=12YB0YDkmLtGjqBcx400HuCSyYnvVXgj8
To: /mnt/disks/user/project/data.pickle
100%|███████████████████████████████████████| 77.1M/77.1M [00:00<00:00, 183MB/s]


In [ ]:
# !pip3 install pickle5
# import pickle5 as pickle
import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG

import pickle
rfile = "data.pickle"
with open(rfile, 'rb') as handle:
    train = pickle.load(handle)

train = [[row[1],row[2],row[3]] for row in train]
test = train[-4000:] ## already shuffled
train = train[0:len(train)-4000]
val = train[-10000:] 
train = train[0:len(train)-10000]
len(train),len(val),len(test)

(90097, 10000, 4000)

In [ ]:
# import pandas as pd
# rfile = "drive/MyDrive/Colab Notebooks/train.csv"
# rData = pd.read_csv(rfile)
# rData = rData.iloc[:,2].tolist()

# # rData = rData[0:10000] ######### take 10%

# import nltk,random
# nltk.download('punkt')
# from tqdm import tqdm
# train = []
# #positives
# for review in tqdm(rData):
#     sent = nltk.sent_tokenize(review)
#     train.append([random.choice(sent),review,1])

# #negatives
# for review in tqdm(rData):
#     random_review = random.choice(rData)
#     sent = nltk.sent_tokenize(random_review)
#     train.append([random.choice(sent),review,0])


# random.shuffle(train)


(100097, 4000)

In [ ]:
# !pip install --upgrade pip
# !pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, models, evaluation

from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator,CEBinaryClassificationEvaluator, CEBinaryAccuracyEvaluator

model = CrossEncoder('sentence-transformers/all-MiniLM-L6-v2', num_labels=1)  ###
# model = CrossEncoder('cross-encoder/nli-deberta-v3-xsmall', num_labels=2)
# model = CrossEncoder('sentence-transformers/all-MiniLM-L6-v2', num_labels=2)
# model = CrossEncoder('cross-encoder/nli-deberta-v3-xsmall', num_labels=2)
# cross-encoder/nli-deberta-v3-xsmall
# cross-encoder/ms-marco-MiniLM-L-12-v2
# https://huggingface.co/models?sort=downloads&search=cross-encoder%2F
# cross-encoder/stsb-roberta-large
# cross-encoder/quora-distilroberta-base

# scores = model.predict([('A man is eating pizza', 'A man eats something'), ('A black race car starts up in front of a crowd of people.', 'A man is driving down a lonely road.')])

#Convert scores to labels
# label_mapping = ['contradiction', 'entailment', 'neutral']
# labels = [label_mapping[score_max] for score_max in scores.argmax(axis=1)]


## Step 1: use an existing language model
# word_embedding_model = models.Transformer('microsoft/deberta-v3-xsmall')
# word_embedding_model = models.Transformer('sentence-transformers/all-MiniLM-L6-v2')

## Step 2: use a pool function over the token embeddings
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

## Join steps 1 and 2 using the modules argument
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/mnt/disks/user/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/mnt/disks/user/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and gui

In [ ]:
from sentence_transformers import InputExample, losses
from torch.utils.data import DataLoader


train_examples = []
for i in range(len(train)):
  train_examples.append(InputExample(texts=[train[i][0], train[i][1]], label=train[i][2])) ### not int


## greater batch better
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
#### not used ???
# train_loss = losses.SoftmaxLoss(model=model,sentence_embedding_dimension=384,num_labels=2)  ## OR .ContrastiveLoss
num_epochs = 10 
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data
# CEBinaryClassificationEvaluator  ###### should works for 0,1
evaluator = CECorrelationEvaluator([[row[0],row[1]] for row in val], 
                [row[2] for row in val])

evaluatorAll = CECorrelationEvaluator([[row[0],row[1]] for row in train], 
                [row[2] for row in train])

evaluatortest = CECorrelationEvaluator([[row[0],row[1]] for row in test], 
                [row[2] for row in test])



In [ ]:
evaluator(model)

-0.0543511070468631

In [ ]:
evaluatortest(model)

INFO:sentence_transformers.cross_encoder.evaluation.CECorrelationEvaluator:CECorrelationEvaluator: Evaluating the model on  dataset:
INFO:sentence_transformers.cross_encoder.evaluation.CECorrelationEvaluator:Correlation:	Pearson: 0.0004	Spearman: -0.0021


-0.002125256946879608

In [ ]:
def testModel(score, epoch, steps):
    print(evaluatortest(model))
    print(score, epoch, steps)

In [ ]:
response = model.fit(train_dataloader=train_dataloader,
          epochs=num_epochs,
          warmup_steps=warmup_steps, 
          evaluator=evaluator, 
          evaluation_steps=500,
            callback = testModel,
          output_path="models")



# evaluator2 = evaluation.BinaryClassificationEvaluator ([row[0] for row in train[0:10000]], 
#                 [row[1] for row in train[0:10000]], 
#                 [row[2] for row in train[0:10000]], 
#                 show_progress_bar = True)
# evaluator2(model)#0.9179824272299182

I0914 16:51:05.407576 140604938979136 tpu_cluster_resolver.py:35] Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
I0914 16:51:05.868639 140604938979136 __init__.py:415] $HOME=/home/ubuntu
I0914 16:51:05.869856 140604938979136 __init__.py:415] CONFIGDIR=/home/ubuntu/.config/matplotlib
I0914 16:51:05.870787 140604938979136 __init__.py:415] matplotlib data path: /mnt/disks/user/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data
I0914 16:51:05.875625 140604938979136 __init__.py:1093] loaded rc file /mnt/disks/user/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc
I0914 16:51:05.877369 140604938979136 __init__.py:1819] matplotlib version 3.0.3
I0914 16:51:05.877919 140604938979136 __init__.py:1820] interactive is False
I0914 16:51:05.878815 140604938979136 __init__.py:1821] platform is linux
I0914 16:51:05.879868 140604938979136 __init__.py:1822] loaded modules: ['sys', 'builtins', '_f

In [ ]:
evaluator(model)


0.9666181938058408

In [ ]:
evaluatortest(model)

0.10241470481115691

In [ ]:
evaluatorAll(model)

In [ ]:
i=300
model.predict([train[i][0],train[i][1]]),train[i][2]

(0.00033773977, 0)

In [ ]:
model.predict(["The college placement ","The college has 100% placement record. The highest package offered is 1.25 Crore per annum, and the lowest package is 4 LPA, and the average package is around 12-15 LPA. The top recruiter is Goldman Sachs. About 30% of students got internships. But the internships for branches like CSE, IT, etc., is more than 80%. Students also got PPO offers. Software developer is the top role offered."])
## add min limit in sent rand step, and in case < 5 words match, add as 0, 


0.95272636

In [ ]:
# https://colab.research.google.com/github/UKPLab/sentence-transformers/blob/master/examples/applications/retrieve_rerank/retrieve_rerank_simple_wikipedia.ipynb